In [ ]:
# imports

# external modules
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# framework modules
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram

# local modules
import omstools
importlib.reload(omstools)

In [ ]:
# define file

# cluster position in xy coordinates
#fname = '../data/data/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1_preprocessed.parquet'

# cluster occupancy
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1_preprocessed.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2_preprocessed.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3_preprocessed.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4_preprocessed.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXForward-clusters_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_1_preprocessed.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXForward-clusters_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_2_preprocessed.parquet'

# digi occupancy
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1_preprocessed.parquet'
fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2_preprocessed.parquet'

In [ ]:
df = pd.read_parquet(fname)
nhists = len(df)
xbins = df['Xbins'][0]
ybins = df['Ybins'][0]
hists = np.array([df['histo'][i].reshape(xbins,ybins) for i in range(nhists)])
runs = np.array(df['fromrun'])
lumis = np.array(df['fromlumi'])
entries = np.array(df['entries'])
print('Shape of hists array: {}'.format(hists.shape))
print('Runs: {}'.format(runs))
print('Lumis: {}'.format(lumis))
print('Entries: {}'.format(entries))

In [ ]:
# plot preprocessed histograms

nplots = 5
plotids = np.random.choice(nhists, size=nplots)

for i in plotids:
    fig, ax = plot_histogram(hists[i])
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# check if normalization is flat with respect to pileup

omsfile = 'omsdata/omsdata_Run2024B-v1.json'
with open(omsfile, 'r') as f:
    omsjson = json.load(f)
    
pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, omsjson, 'pileup')
normalized_entries = np.sum(np.sum(hists, axis=-1), axis=-1)

fig,ax = plt.subplots()
ax.scatter(pileup, normalized_entries, s=1, alpha=0.25, c='b')
ax.set_ylim((0, 5e5))
ax.set_xlabel('Pileup', fontsize=15)
ax.set_ylabel('Occupancy', fontsize=15)